In [83]:
import numpy as np
import galois
import itertools
from scipy import sparse
GF = galois.GF(2)

In [84]:
# https://scialert.net/fulltext/?doi=tasr.2012.929.934
def girth(H, g):
    # girth can only be even in bipartite graphs
    combs = list(itertools.combinations(np.arange(H.shape[0]), g//2))
    for c in combs:
        w = np.zeros(H.shape[1])
        for r in range(g//2):
            w += np.array(H[c[r]])
        if (np.count_nonzero(w == 2) == g//2):
            return True
    return False

In [134]:
# configuration model
# https://en.wikipedia.org/wiki/Configuration_model

n = 16
deg_c = 4 # w_r. Every check has this many bits in it, d
deg_v = 3 # w_c. Every bit is in this many checks, c
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), np.int64)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            # if (girth(H, 4)):
            #     H[cs[c_ind]][vs[v_ind]] = 0
            #     break
            vs = np.delete(vs, v_ind)
            cs = np.delete(cs, c_ind)

# H = sparse.csc_matrix(H)
H = GF(H)

In [135]:
# while (girth(H, 4)):
#     arange = np.arange(H.shape[0])
#     i = np.random.choice(arange, replace=False)
#     j = np.random.choice(arange, replace=False)
#     bits_i = np.where(H[i])[0]
#     bits_j = np.where(H[j])[0]
#     i_ind = np.random.choice(bits_i)
#     j_ind = np.random.choice(bits_j)
    
#     if (i_ind == j_ind or i_ind in bits_j or j_ind in bits_i): 
#         continue
    
#     v = GF(np.zeros(H.shape[1], dtype=np.int8))
#     v[i_ind] = v[j_ind] = 1
#     H[i] += v
#     H[j] += v

In [136]:
def rref(H):
    r = 0
    for c in range(k, n):
        if (H[r, c] == 0):
            # need to swap
            for r2 in range(r+1, num_checks):
                if (H[r2, c] == 1):
                    temp = H[r, :].copy()
                    H[r, :] = H[r2, :]
                    H[r2, :] = temp
                    break
                    
        if (H[r, c] == 0):
            print('H is singular')

        # cancel out other rows
        for r2 in range(r+1, num_checks):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] + H[r, :]

        # # back substitution
        for r2 in range(0, r):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] + H[r, :]

        r += 1
    return H

rref_H = GF(rref(GF(H.copy())))

G = GF(np.hstack((GF(np.eye(k, dtype=np.int64)), rref_H[:,0:k].T)))
print(np.any(H @ G.T))

False


In [137]:
H

GF([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0],
    [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
    [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]], order=2)

In [27]:
for i in range(2**k):
    m = [int(j) for j in bin(i)[2:].zfill(k)]
    c = GF(m) @ G
    print(np.array(c), np.count_nonzero(c))

[0 0 0 0 0 0] 0
[0 1 1 0 1 0] 3
[1 0 0 1 1 0] 3
[1 1 1 1 0 0] 4


In [564]:
def flip_alt3(y, H):
    def determine_flippable(w):
        flippable = []
        for v in range(H.shape[1]):
            checks = np.nonzero(H[:, v])[0]
            unsatisfied = np.sum([x(j, w) for j in checks])
            deg = len(checks)
            if (2 * unsatisfied >= deg):
                flippable.append(v)
        return flippable

    # evaluates the lth parity check with recieved vector y
    x = lambda l, y: np.sum([y[k] for k in np.where(H[l])]) % 2 

    w = y.copy()
    flippable = determine_flippable(w)
    while flippable:
        z = GF(np.zeros(H.shape[1], dtype=np.int8))
        z[flippable[0]] = 1
        w += z
        flippable = determine_flippable(w)

    if np.any(H @ w):
        return "FAIL"
    else:
        # where w is the deduced word
        return w

In [565]:
indices = [set(H[:,i].indices) for i in range(H.shape[1])]

In [593]:
def flip(e, H, B):
    # e is the error
    # H is the parity check matrix
    # B is the number by which the syndrome has to decrease each iteration 
    sigma = set(np.nonzero(H.dot(e) % 2)[0])
    e = set()

    while sigma:
        valid = False
        for i in range(H.shape[1]):
            # loop through all variables
            neighborhood = indices[i]
            new_sigma = sigma ^ neighborhood

            if len(new_sigma) <= len(sigma) - B:
                e = e ^ set([i])
                sigma = new_sigma
                valid = True
                break

        if (not valid):
            break
    return e

In [567]:
def flip_alt2(y, H):
    w = y.copy()
    syndrome = H.dot(w) % 2

    while True:
        terminate = True
        for v in range(H.shape[1]):
            checks = indices[v]
            unsatisfied = syndrome[checks]
            if (2 * np.sum(unsatisfied) > deg_v):
                if (w[v]):
                    np.put(w, v, [0])
                else:
                    np.put(w, v, [1])

                np.put(syndrome, checks[unsatisfied == 0], [1])
                np.put(syndrome, checks[unsatisfied == 1], [0])
                terminate = True
                break
        if (not terminate):
            continue
        
        if np.any(H @ w):
            return "FAIL"
        else:
            return w

In [632]:
p = 0.1
# num_errors = 100
sum = 0
num_runs = 1

for i in range(num_runs):
    # e = np.array([1] * num_errors + [0] * (n - num_errors))
    e = [1 if np.random.uniform() < p else 0 for i in range(n)] 
    np.random.shuffle(e)
    e_hat = flip(e, H, 1)

    if (set(np.where(e)[0]) == e_hat):
        sum += 1

In [633]:
sum / num_runs

1.0

In [634]:
len(e_hat ^ set(np.where(e)[0]))

0